# Fundamental diagrams of pedestrian flow from empirical video data

## Author
Étienne Beauchamp  
M.A.Sc. candidate, Department of Civil, Geological and Mining Engineering  
Polytechnique Montréal, C.P. 6079, succ. Centre-Ville  
Montréal (Québec) Canada H3C 3A7  
Phone: (514) 340-4711 ext. 4210  
Email: etienne-2.beauchamp@polymtl.ca

## Overview
This code was written for a project done as part of the course *CIV6705 - Théorie de la circulation*. It uses methods from the TrafficIntelligence Python module to draw fundamental diagrams of pedestrian data extracted from video recordings.

Before getting started, you will need to install the following:
* Python 3.5.6 (other versions were not tested) with most of the standard scientific packages (e.g., numpy, matplotlib, pandas, etc.). I recommend installing [Anaconda](https://www.anaconda.com/download/#linux);
* [TrafficIntelligence](https://bitbucket.org/Nicolas/trafficintelligence/wiki/Home);
* [OpenCV 3.4.3](https://anaconda.org/conda-forge/opencv).

In [13]:
from trafficintelligence import storage, moving
from scipy.spatial import Voronoi, voronoi_plot_2d

import numpy as np
import matplotlib.pyplot as plt

objects = storage.loadTrajectoriesFromSqlite('Donnees/1014AM_July 11_2014(E to W).sqlite','feature')

frameRate = 30

### Voronoï diagram technique